# Evaluation (II)
## Training the SVM and Compare Results

In [1]:
import os
import sys
import json
import pandas as pd
sys.path.append("../")
from rs_helper import DAN,FastText, scatter_plot

In [2]:
def read_data(_dir:str) -> pd.DataFrame:
    data = {}
    #data["url"] = []
    data["text"] = []
    data["class"] = []
    for root, dirs, files in os.walk(_dir):
        for _dir in dirs: 
            for txt_file in [x for x in os.listdir(os.path.join(root, _dir)) if x.endswith((".txt", ".TXT"))]:
                # Class name = dir name
                class_name = _dir
                #Read File
                file_name = os.path.abspath(os.path.join(root, _dir, txt_file))
                file = open(file_name, "r")
                txt = file.read()
                file.close()
                #data["url"].append(file_name)
                data["text"].append(txt)
                data["class"].append(class_name)
    df = pd.DataFrame.from_dict(data)
    del data
    return df

In [3]:
data_dir = "../datasets/final_datasets/clean_dataset_equal"

In [4]:
df = read_data(data_dir).sample(frac=1)

In [5]:
df_eval = read_data("../datasets/eval_data/")

## Add Embeddings

In [6]:
ft_model = FastText()

INFO:tensorflow:FastText Model is loading


INFO:tensorflow:FastText Model is loading


INFO:tensorflow:FastText Model loaded!


INFO:tensorflow:FastText Model loaded!


In [7]:
#frozen_graph_path = "TEST_CLEAN_EQUAL/frozen_graph.pb"

In [8]:
#dan = DAN(ft_model,frozen_graph_path)

In [9]:
path = "DAN_COMBINATIONS"

In [10]:
dan_models = []

In [11]:
for model in os.listdir(path):
    if os.path.isdir(os.path.join(path,model)):
        if model != ".ipynb_checkpoints":
            frozen_graph = os.path.join(path, model, "frozen_graph.pb")
            config = os.path.abspath(os.path.join(path,model, "config.json"))
            dan_models.append((DAN(ft_model, frozen_graph), config))

## Train Test Split

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [14]:
counter = 1
for dan, config in dan_models[24:]:
    # handling df and embeddings
    _df = df
    _df_eval = df_eval
    _df["embedding"] = _df["text"].apply(lambda x: dan.inference(x)[0])
    _df_eval["embedding"] = _df_eval["text"].apply(lambda x: dan.inference(x)[0])
    
    x,y = train_test_split(_df, test_size = 0.1, shuffle = True)
    
    # define svc
    svc_linear = SVC(gamma = "scale", kernel = "linear", C=12)
    svc_linear.fit(_df["embedding"].tolist(), _df["class"].tolist())
    
    svc_rbf = SVC(gamma = "scale", kernel = "rbf", C=12)
    svc_rbf.fit(_df["embedding"].tolist(), _df["class"].tolist())
    
    insampling_score_l = svc_linear.score(x["embedding"].tolist(), x["class"].tolist())
    test_score_l = svc_linear.score(y["embedding"].tolist(), y["class"].tolist())
    valid_score_l = svc_linear.score(_df_eval["embedding"].tolist(),_df_eval["class"].tolist())
    
     # define svc
    svc_rbf = SVC(gamma = "scale", kernel = "rbf", C=12)
    svc_rbf.fit(_df["embedding"].tolist(), _df["class"].tolist())
    insampling_score_r = svc_rbf.score(x["embedding"].tolist(), x["class"].tolist())
    test_score_r = svc_rbf.score(y["embedding"].tolist(), y["class"].tolist())
    valid_score_r= svc_rbf.score(_df_eval["embedding"].tolist(),_df_eval["class"].tolist())
    
    # Read Score
    with open(config, "r") as _json:
            _dict = json.load(_json)
            
    _dict["insampling_linear_kernel"] = insampling_score_l
    _dict["test_linear_kernel"] = test_score_l
    _dict["valid_score_linear_kernel"] = valid_score_l
    
    _dict["insampling_rbf_kernel"] = insampling_score_r
    _dict["test_rbf_kernel"] = test_score_r
    _dict["valid_score_rbf_kernel"] = valid_score_r
    
    print(_dict)
    # Write new JSON
    with open(config, "w") as _json:
        json.dump(_dict, _json)
    
    print("{} of {}".format(counter, len(dan_models)))
    counter+=1